In [14]:
import os
import torch

import utils

print(os.listdir("../data/"))

['kawiki.txt', 'data.txt', 'data2.txt']


In [27]:
from utils import GeorgianLanguageDatasetLoader
import gc
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = GeorgianLanguageDatasetLoader("../data/kawiki.txt", 20, device)

In [28]:
vocab = dataset.get_vocabulary()
print(vocab.get_itos()[:20])

['<unk>', '<bos>', '<eos>', ',', '.', 'და', ')', '(', '„', '“', '—', 'წელს', 'იყო', 'წლის', ':', 'რომელიც', 'შემდეგ', 'ამ', 'საქართველოს', 'რომ']


In [29]:
train_data, valid_data, test_data = dataset.get_data()
train_data[:10]

tensor([ 9214, 54332, 33110,     3,    56,    86,  5227,  6449, 74311,     3])

In [30]:
' '.join([vocab.get_itos()[i] for i in train_data[40:60]])

'საუკუნეებში მას „ ოძრხედ “ მოიხსენიებენ . მთავარი კერძებიდან ერთ-ერთი ყველაზე პოპულარული კერძი , რომელიც დელიკატესად ითვლება არის მოხარშული ცხვრის'

In [31]:
text_pipeline = dataset.get_text_pipeline()
text_pipeline("შავი კაცი მიდიოდა")

[341, 549, 4165]

In [32]:
train_data_batched, val_data_batched = dataset.get_batched_data()

In [33]:
train_data_batched.shape, train_data_batched.device

(torch.Size([20, 29316]), device(type='cuda', index=0))

In [34]:
x, y = utils.get_batch(train_data_batched, 1)

In [35]:
x.shape, y.shape, train_data_batched.shape

(torch.Size([20, 10]), torch.Size([20, 10]), torch.Size([20, 29316]))

In [36]:
x.storage().data_ptr() == train_data_batched.storage().data_ptr()

True

In [37]:
x[0], y[0]

(tensor([54332, 33110,     3,    56,    86,  5227,  6449, 74311,     3,    44],
        device='cuda:0'),
 tensor([33110,     3,    56,    86,  5227,  6449, 74311,     3,    44,  6864],
        device='cuda:0'))

In [43]:
model = utils.LSTMModel(50, 300, len(vocab), device, 1).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched)

RuntimeError: CUDA out of memory. Tried to allocate 104.00 MiB (GPU 0; 1.96 GiB total capacity; 1.43 GiB already allocated; 14.69 MiB free; 1.45 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload
text = utils.generate_text(model,device, dataset.vocab, 'იკვლევდა ევროპის და განსაკუთრებით', 20)

In [ ]:
print(text)